In [31]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import binom, betaln
from scipy.stats import beta
from scipy.optimize import fmin 
import sys

### Introduction

Implementation of the minimum number of samples needed for acheive a specific HDI width, given psuedo variables.

In [30]:
def HDIofICDF(dist_name, credMass=0.95, **args):
    # freeze distribution with given arguments
    distri = dist_name(**args)
    # initial guess for HDIlowTailPr
    incredMass =  1.0 - credMass

    def intervalWidth(lowTailPr):
        return distri.ppf(credMass + lowTailPr) - distri.ppf(lowTailPr)

    # find lowTailPr that minimizes intervalWidth
    HDIlowTailPr = fmin(intervalWidth, incredMass, ftol=1e-8, disp=False)[0]
    # return interval as array([low, high])
    return distri.ppf([HDIlowTailPr, credMass + HDIlowTailPr])

In [32]:
def minNforHDIPower(
    gen_prior_mean, 
    gen_prior_n,
    hdi_max_width=None,
    null_val=None,
    rope=None,
    desired_power=0.8,
    aud_prior_mean=0.5,
    aud_prior_n = 2,
    hdi_mass=0.95,
    init_sample_size=1,
    verbose=True
):
    if hdi_max_width != None and null_val != None:
        print("Only one of hdi_max_width or null_val should be specified")
        sys.exit(0)
    if rope == None:
        rope = [null_val, null_val]
    gen_prior_a = gen_prior_mean * gen_prior_n
    gen_priob_b = (1.0 - gen_prior_mean) * gen_prior_n
    
    aud_prior_a = aud_prior_mean * aud_prior_n
    aud_prior_b = (1.0 - aud_prior_mean) * aud_prior_n
    
    sample_size = init_sample_size
    
    # Increment sample size till power is acheived
    while True:
        # All possible values of z i.e all possible number of z heads possible in N flips.
        zvec = np.arange(0, sample_size + 1)
        
        # Compute probability of each z value for the data-generating prior
        pzvec = np.exp(np.log(binom(sample_size, zvec))
                + betaln(zvec + gen_prior_a, sample_size - zvec + gen_priob_b)
                - betaln(gen_prior_a, gen_priob_b))
        
        hdi_mat = np.zeros((len(zvec), 2))
        
        for z_idx, z_val in enumerate(zvec):
            hdi_mat[z_idx] = HDIofICDF(
                beta, 
                credMass=hdi_mass,
                a=(z_val + aud_prior_a),
                b=(sample_size - z_val + aud_prior_b)
            )
            
        power_hdi = 0
        if hdi_max_width != None:
            hdi_width = hdi_mat[:, 1] - hdi_mat[:, 0]
            power_hdi = np.sum(pzvec[hdi_width < hdi_max_width])
        
        if null_val != None:
            power_hdi = np.sum(pzvec[(hdi_mat[:, 0] > rope[1]) | (hdi_mat[:, 1] < rope[0])])
            
        if verbose:
            print("For sample size {} \n Power: {}".format(sample_size, power_hdi))
        
        if power_hdi > desired_power:
            break
        else:
            sample_size += 1
    return sample_size

In [35]:
minNforHDIPower(gen_prior_mean=0.85, gen_prior_n=20, null_val=0.5, desired_power=0.95, verbose=True)

For sample size 1 
 Power: 0.0
For sample size 2 
 Power: 0.0
For sample size 3 
 Power: 0.0
For sample size 4 
 Power: 0.5488424618859402
For sample size 5 
 Power: 0.4792490118577074
For sample size 6 
 Power: 0.4214624505928853
For sample size 7 
 Power: 0.7137549407114623
For sample size 8 
 Power: 0.6628600385122122
For sample size 9 
 Power: 0.6153568170959478
For sample size 10 
 Power: 0.7918803834845813
For sample size 11 
 Power: 0.7549442311811129
For sample size 12 
 Power: 0.7186577926917433
For sample size 13 
 Power: 0.8364234933224627
For sample size 14 
 Power: 0.8085098579742385
For sample size 15 
 Power: 0.7803561562327331
For sample size 16 
 Power: 0.8647358139112257
For sample size 17 
 Power: 0.8428173743040511
For sample size 18 
 Power: 0.9011455574424082
For sample size 19 
 Power: 0.884098467529333
For sample size 20 
 Power: 0.8663361553626093
For sample size 21 
 Power: 0.9121766612867831
For sample size 22 
 Power: 0.8980626980339392
For sample size 23 
 

31